In [ ]:
!pip install import-ipynb

Importação das váriveis pré tranformadas

In [9]:
import seaborn as sns 
import matplotlib_inline as plt
import import_ipynb
from carregar import df_poluicao_osasco, df_sintomas_osasco

In [10]:
print('Dados de Poluição:',df_poluicao_osasco.head())
print('Dados de sintomas:',df_sintomas_osasco.head())

Dados de Poluição:           date  pm25  pm10  o3  no2  so2  co
431 2024-01-01    33    20        7    1   2
261 2024-01-02    54    18       12        3
262 2024-01-03    54    19       12        4
263 2024-01-04    37    10        7        3
264 2024-01-05    20    13       10        4
Dados de sintomas:       DT_NOTIFIC  FEBRE  TOSSE  GARGANTA  DISPNEIA  DESC_RESP  SATURACAO  \
872   25/09/2024    2.0    2.0       2.0       1.0        1.0        1.0   
1231  11/07/2024    2.0    1.0       2.0       2.0        1.0        2.0   
1495  22/09/2024    1.0    2.0       2.0       1.0        1.0        1.0   
1497  22/08/2024    1.0    1.0       2.0       2.0        1.0        2.0   
2191  23/05/2024    2.0    1.0       2.0       1.0        2.0        2.0   

      DIARREIA  VOMITO  DOR_ABD  FADIGA  PERD_OLFT  PERD_PALA  OUTRO_SIN  \
872        2.0     2.0      2.0     2.0        2.0        2.0        2.0   
1231       2.0     2.0      2.0     2.0        2.0        2.0        1.0   
1495   